In [11]:
import pandas as pd
import numpy as np
import numpy.random
import os
import gc
import h5py
from collections import deque
import cv2

import sklearn
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, roc_curve, auc

from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation

%matplotlib inline

## Load data

In [2]:
citydata = pd.read_csv('Data/CityData.csv')

In [3]:
citydata

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234
4,4,236,241
5,5,315,281
6,6,358,207
7,7,363,237
8,8,423,266
9,9,125,375


Axes: Days / Hours / Models / x / y

In [4]:
# read h5 format back to numpy array
# For file creation cf Hackathon tutorial notebook
h5f = h5py.File('Data/METdata.h5', 'r')
train = h5f['train'][:]
test = h5f['test'][:]
h5f.close()

In [5]:
train.shape, test.shape

((5, 18, 11, 548, 421), (5, 18, 10, 548, 421))

# Data preparation

In [6]:
nTrainDays = 5
nValidationDays = 0

useModels = [0,1,2,3,4,5] 

In [7]:
plt.figure(figsize=(20,10))
plt.imshow(train[nTrainDays,0,10,:,:].T>15)
plt.savefig('true.png')

IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
trainData = train[:nTrainDays,:,useModels+[10],:,:]
trainData = np.rollaxis(trainData,2,5)
trainData = trainData.reshape(-1,trainData.shape[-1])
x_train = trainData[:,0:-1]
y_train = trainData[:,-1]

valData = train[nTrainDays:(nValidationDays+nTrainDays),:,useModels+[10],:,:]
valData = np.rollaxis(valData,2,5)
valData = valData.reshape(-1,trainData.shape[-1])
x_val = valData[:,0:-1]
y_val = valData[:,-1]

print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

del train, trainData, valData

In [ ]:
gc.collect()

## Prediction

In [ ]:
#model = LogisticRegression()
model = XGBClassifier(n_estimators=100)
#model = SVC(kernel='linear', verbose=True, max_iter=50, probability=True)
#model = KNeighborsClassifier(n_neighbors=10, n_jobs=3) # Very good results but very noisy

thresh = 0.5

In [ ]:
#red = PCA(3)
#red.fit(x_train)
#x_train = red.transform(x_train)
#x_val = red.transform(x_val)

In [ ]:
#print(red.explained_variance_ratio_)
#del red

In [ ]:
y_train

In [ ]:
model.fit(x_train, y_train>15)

In [ ]:
pred_val = model.predict_proba(x_val)[:,1]
pred_train = model.predict_proba(x_train)[:,1]

In [ ]:
fpr, tpr, _ = roc_curve(y_val>15, pred_val)
roc_auc = auc(fpr, tpr)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#pred_val = np.amax(x_val, axis=1)
#pred_train = np.amax(x_train, axis=1)

In [ ]:
pred_val.shape, pred_train.shape

In [ ]:
indices = np.random.choice(pred_train.shape[0], 5000)

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(10,5), sharex=True, sharey=True)

ax1.scatter(pred_train[indices], y_train[indices])
ax2.scatter(pred_val[indices], y_val[indices])

ax1.axhline(15, color='r')
ax2.axhline(15, color='r')

ax1.axvline(thresh, color='r')
ax2.axvline(thresh, color='r')

ax1.set_xlabel('Predicted probability (training data)')
ax2.set_xlabel('Predicted probability (validation data)')

ax1.set_ylabel('Actual wind speed')

ax1.text(0.25,5, 'TN', color='r', size=30,horizontalalignment='center')
ax1.text(0.25,25, 'FN', color='r', size=30,horizontalalignment='center')
ax1.text(0.75,25, 'TP', color='r', size=30,horizontalalignment='center')
ax1.text(0.75,5, 'FP', color='r', size=30,horizontalalignment='center')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_val>=15, pred_val>0.5).ravel()
fn/pred_val.shape[0]

In [ ]:
tp,fp,fn,tp

In [ ]:
predMatrix = pred_val.reshape(nValidationDays,18,548,421,1)
trueMatrix = y_val.reshape(nValidationDays,18,548,421)

In [ ]:
trueMatrix.shape

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax.imshow(np.logical_xor(trueMatrix[0, 0, :, :].T > 15, predMatrix[0, 0, :, :, 0].T > thresh))
ax.set_xlabel('Yellow: wrong prediction, purple: right prediction')

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(20,20))

ax1.imshow(trueMatrix[0,0,:,:].T>15)
ax2.imshow(predMatrix[0,0,:,:,0].T>thresh)

ax1.set_xlabel('True data')
ax2.set_xlabel('Prediction')


## Search on the graph

### Prepare data

In [ ]:
max_x = 548
mazx_y = 421

In [ ]:
day = 0
predDay = predMatrix[day,:,:,:,:]
trueDay = trueMatrix[day,:,:,:]
predDay.shape, trueDay.shape

In [ ]:
start_city = 0
start_x = citydata.iloc[start_city,1]
start_y = citydata.iloc[start_city,2]
start_x, start_y

In [ ]:
obj_city = 1
obj_x = citydata.iloc[obj_city,1]
obj_y = citydata.iloc[obj_city,2]
obj_x, obj_y

### Define BFS algorithm for search

In [ ]:
def BFS(grid,start, goal, block=[1],mode='pos',goalType=None, typeGrid=None):
    res = np.zeros(grid.shape) - 1
    res[start] = 0
    stack = queue = deque([start])
    found=False
    
    if goal[0]>=start[0]:
        if goal[1]>=start[1]:
            dirs = [(1,0),(0,1),(-1,0),(0,-1)]
        else:
            dirs= [(1,0),(0,-1),(-1,0),(0,1)]
    else:
        if goal[1]>=start[1]:
            dirs = [(-1,0),(0,1),(1,0),(0,-1)]
        else:
            dirs=[(-1,0),(0,-1),(1,0),(0,1)]
    while len(stack)>0 and not found:
        new = stack.popleft()
        for d in dirs:
            try_new = (new[0]+d[0], new[1]+d[1])
            if (try_new[0]>=0) and (try_new[0]<grid.shape[0]) and(try_new[1]>=0) and (try_new[1]<grid.shape[1]):
                if grid[try_new] not in block:
                    if res[try_new] == -1:
                        res[try_new] = res[new] +1
                        if mode=='pos' and try_new[0]==goal[0] and try_new[1]==goal[1]:
                            found=True
                            break
                        if mode=='type' and typeGrid[try_new==goalType]:
                            found=True
                            break
                        stack.append(try_new)
                        
    return res, found           

In [ ]:
def backTrackBFS(grid, start,goal, block, mode, goalType=None, typeGrid=None):
    res, found = BFS(grid,start,goal,block, mode, goalType, typeGrid)
    
    if goal[0]>=start[0]:
        if goal[1]>=start[1]:
            dirs = [(1,0),(0,1),(-1,0),(0,-1)]
        else:
            dirs= [(1,0),(0,-1),(-1,0),(0,1)]
    else:
        if goal[1]>=start[1]:
            dirs = [(-1,0),(0,1),(1,0),(0,-1)]
        else:
            dirs=[(-1,0),(0,-1),(1,0),(0,1)]
    
    if found:
        posList = [goal]
        current = goal
        currentDist = res[current]
    else:
        accessibleX, accessibleY = np.where(res!=-1)
        bestDist = 10000
        bestPos = None
        for i in range(accessibleX.shape[0]):
            tryPos = (accessibleX[i], accessibleY[i])
            dist = abs(tryPos[0]-goal[0]) + abs(tryPos[1]-goal[1])
            if dist<bestDist:
                bestDist=dist
                bestPos=tryPos
        current=bestPos
        currentDist= res[current]
        posList = [current]
        
    while currentDist != 0:
        for d in dirs:
            try_new = (current[0]+d[0], current[1]+d[1])
            if (try_new[0]>=0) and (try_new[0]<grid.shape[0]) and(try_new[1]>=0) and (try_new[1]<grid.shape[1]):
                if res[try_new] == currentDist-1:
                    posList.append(try_new)
                    current=try_new
                    currentDist = currentDist-1
                    break
    return list(reversed(posList)), found

### Perform search

In [ ]:
predDay.shape

In [ ]:
start = start_x, start_y
goal = obj_x, obj_y

mindless_algo_time=0
thresh = thresh

def getPath(start, goal, mindless_algo_time, predDay, thresh):
    predDayThresh = predDay>thresh
    pos=start
    fullpath=[]
    for hour in range(18):
        print('Hour',hour)
        currentGrid = predDayThresh[hour,:,:,0]
        if mindless_algo_time>0:
            if hour==17:
                mindless_algo_time=30
            # Move towards the city only with the data from this hour
            path, found = backTrackBFS(currentGrid,pos,goal,[True],'pos')
            if len(path)>mindless_algo_time:
                path=path[:mindless_algo_time]
            elif len(path)<mindless_algo_time:
                path= path + [path[-1] for i in range(mindless_algo_time-len(path))]

            fullpath = fullpath+path
            pos = path[-1]
            if pos==goal:
                break
            #print(len(path))
            elapsed = len(path)
            # Move towards the city, but counting blocking points from both this hour and the next
            if hour==17:
                break
            escapeGrid = predDayThresh[hour+1,:,:,0]
            if escapeGrid[pos]:
                print('Escaping')
                # In that case we will be in a turbulence in the next hour. We move towards a safe zone and resume
                escapePath, _ = backTrackBFS(currentGrid, pos, goal, [True], 'type', goalType=False, typeGrid=escapeGrid)
                fullpath = fullpath + escapePath
                elapsed+= len(escapePath)
                pos = fullpath[-1]
                if pos==goal:
                    break
            if elapsed>30:
                print("Boom you're dead")
                break
        else:
            elapsed=0
            escapeGrid = predDayThresh[hour+1,:,:,0]
        currentGrid = np.logical_and(currentGrid,escapeGrid)
        path, found = backTrackBFS(currentGrid,pos,goal,[True],'pos')
        #print(len(path))
        #print('Elapsed:',elapsed)
        if len(path)>(30-elapsed):
            path=path[:(30-elapsed)]
        elif len(path)<(30-elapsed):
            print('path of length',len(path), 'filling to 30')
            path = path + [path[-1] for i in range(30-elapsed - len(path))]
            
        #print(len(path))
        fullpath = fullpath+path
        pos = fullpath[-1]
        if pos==goal:
            break
        print('full length:',len(fullpath))
    return fullpath

In [ ]:
start,goal

In [ ]:
fullpath = getPath(start,goal, mindless_algo_time, predDay, thresh)
fullpath

In [ ]:
def toHourMin(step):
    step=step*2
    hour = step//60
    minute = step%60
    return "%02d:%02d" % (hour,minute)

toHourMin(35)

In [ ]:
def getBalloonDay(balloon, day, thresh, ret='np'):
    
    predDay = predMatrix[day,:,:,:,:]
    predDay.shape
    
    start_city = 0
    start_x = citydata.iloc[start_city,1]
    start_y = citydata.iloc[start_city,2]
    start_x, start_y
    
    obj_city = balloon
    obj_x = citydata.iloc[obj_city,1]
    obj_y = citydata.iloc[obj_city,2]
    obj_x, obj_y
    
    start = start_x, start_y
    goal = obj_x, obj_y

    mindless_algo_time=15

    fullpath = getPath(start,goal, mindless_algo_time, predDay, thresh)
    #print(fullpath)
    if ret=='np':
        res = [(balloon,day+6,toHourMin(i),t[0],t[1]) for i,t in enumerate(fullpath)]
        return np.array(res)
    else:
        return fullpath

In [ ]:
predDay.shape

In [ ]:
def showPath(start, goal, path, trueWind,i, figsize=(20,10),figax=None):
    zipped = list(zip(*path))
    xPath = list(zipped[0])
    yPath = list(zipped[1])
    if figax is None:
        fig, ax= plt.subplots(figsize=figsize)
    else:
        fig,ax = figax
        
    ax.plot(xPath[:(30*(i+1))],yPath[:(30*(i+1))], color='green', linewidth=8)
    ax.imshow(trueWind[i,:,:].T)
    ax.plot(goal[0],goal[1], 'ro', markersize=20)
    ax.plot(start[0], start[1],'bo',markersize=20)
    
def showFullPath(start,goal,path,trueWind):
    fig, axs = plt.subplots(nrows=5,ncols=4, figsize=(30,24))
    for y in range(4):
        for x in range(5):
            showPath(start,goal,path, trueWind,4*y+x,None, (fig,axs[x][y]))

showPath(start,goal,fullpath,predDay[:,:,:,0]>15,1)
#showFullPath(start,goal,fullpath,trueDay>15)

In [ ]:
fullpath[]

In [ ]:
days = []
for day in range(5):
    days.append(np.concatenate([getBalloonDay(i+1,day) for i in range(10)]))
    
day=np.concatenate(days)
    

In [ ]:
predDay.shape

In [ ]:
predDay[0,:,:,0]

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(predDay[0,:,:,0]>0.9)

In [ ]:
path = getBalloonDay(1,0)
import matplotlib.animation as animation
fig, ax = plt.subplots()

def

In [ ]:
path

In [ ]:
day[:,0]=(np.array([int(x)+1 for x in day[:,0]]))

In [ ]:
pd.DataFrame(day).to_csv('sub.csv',header=False)

In [ ]:
day